In [1]:
import pandas as pd
import requests
import time
import utils as eda
import os

## Datasets de peliculas y datos de reseñas y etiquetas

In [2]:

print(os.listdir("../data/interim/movies-data/ml-32m"))

['README.txt', 'checksums.txt', 'tags.csv', 'movies.csv', 'links.csv', 'ratings.csv']


In [3]:

informacion = ["links", "movies", "ratings", "tags"]
df_list = []

for i in informacion:
    df = pd.read_csv(f"../data/interim/movies-data/ml-32m/{i}.csv")
    df_list.append(df)


In [4]:
#Links
links_df = df_list[0].head()

##La siguiente información muestra como funciona la info que sale en el dataset de links
"""row = df_list[df_list["movieId"] == 1].iloc[0]

imdb_url = f"https://www.imdb.com/title/tt{int(row['imdbId']):07d}"
tmdb_url = f"https://www.themoviedb.org/movie/{int(row['tmdbId'])}"

print("IMDb:", imdb_url)
print("TMDb:", tmdb_url"""

links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [5]:
#Movies 
movies_df = df_list[1]
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#Raiting
rating_df =  df_list[2].head()
rating_df

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [7]:
#Tags
tags_df = df_list[3].head()
tags_df

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


## Información referente a comentarios de sentimientos, estos deben asociarse a peliculas 

In [8]:
def cargar_reseñas(directorio):
    textos = []
    etiquetas = []

    for label in ['pos', 'neg']:
        path = os.path.join(directorio, label)
        for archivo in os.listdir(path):
            with open(os.path.join(path, archivo), encoding='utf-8') as f:
                textos.append(f.read())
                etiquetas.append(1 if label == 'pos' else 0)
    return textos, etiquetas

# Cargar datos de entrenamiento y prueba
train_texts, train_labels = cargar_reseñas("../data/interim/feeling/aclImdb/train") ##texto de reseñas // ETIQUETA para positivo y negativo
test_texts, test_labels = cargar_reseñas("../data/interim/feeling/aclImdb/test")


## Otro sitio de peliculas

Este servira mas para mostrar datos o complementar los de una pelicula que ya tengamos en el dataset principal

In [9]:
##Esta es para varias peliculas
api_key = "fa1b7162" ##Esto es lo importante, no lo borren ni cambien
peliculas = ["Inception", "The Matrix", "Toy Story"]

datos_peliculas = []

for titulo in peliculas:
    url = f"http://www.omdbapi.com/?t={titulo}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    if data["Response"] == "True":
        datos_peliculas.append(data)

datos_peliculas

[{'Title': 'Inception',
  'Year': '2010',
  'Rated': 'PG-13',
  'Released': '16 Jul 2010',
  'Runtime': '148 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Christopher Nolan',
  'Writer': 'Christopher Nolan',
  'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page',
  'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.',
  'Language': 'English, Japanese, French',
  'Country': 'United States, United Kingdom',
  'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '87%'},
   {'Source': 'Metacritic', 'Value': '74/100'}],
  'Metascore': '74',
  'imdbRating': '8

In [10]:
df = pd.DataFrame(datos_peliculas, columns=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production', 'Website', 'Response'])
print(df)

        Title  Year  Rated     Released  Runtime  \
0   Inception  2010  PG-13  16 Jul 2010  148 min   
1  The Matrix  1999      R  31 Mar 1999  136 min   
2   Toy Story  1995      G  22 Nov 1995   81 min   

                          Genre                         Director  \
0     Action, Adventure, Sci-Fi                Christopher Nolan   
1                Action, Sci-Fi  Lana Wachowski, Lilly Wachowski   
2  Animation, Adventure, Comedy                    John Lasseter   

                                       Writer  \
0                           Christopher Nolan   
1             Lilly Wachowski, Lana Wachowski   
2  John Lasseter, Pete Docter, Andrew Stanton   

                                              Actors  \
0  Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...   
1  Keanu Reeves, Laurence Fishburne, Carrie-Anne ...   
2                  Tom Hanks, Tim Allen, Don Rickles   

                                                Plot  ... Metascore  \
0  A thief who steals cor

Combinación de ambas

In [11]:
movies_df.size

262755

In [12]:
import re
movies = df_list[1].head(50)
# Función para limpiar título
def limpiar_titulo(titulo):
    # Esto elimina " (1995)" u otros años entre paréntesis
    return re.sub(r"\s*\(\d{4}\)$", "", titulo)

# Aplicar limpieza
movies["clean_title"] = movies["title"].apply(limpiar_titulo)

# Extraer año si quieres
def extraer_ano(titulo):
    match = re.search(r"\((\d{4})\)$", titulo)
    if match:
        return int(match.group(1))
    return None

movies["year"] = movies["title"].apply(extraer_ano)

movies.head()

/tmp/ipykernel_6953/3273266103.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["clean_title"] = movies["title"].apply(limpiar_titulo)
/tmp/ipykernel_6953/3273266103.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["year"] = movies["title"].apply(extraer_ano)


,movieId,title,genres,clean_title,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995


In [13]:

omdb_url = "http://www.omdbapi.com/"

# 3️⃣ Función para obtener info de OMDb
def obtener_info_omdb(titulo):
    params = {
        "t": titulo,
        "apikey": api_key,
        "plot": "short",  # puedes poner "full" si quieres la sinopsis completa
        "r": "json"
    }
    response = requests.get(omdb_url, params=params)
    data = response.json()
    if data.get("Response") == "True":
        return {
            "Title": data.get("Title"),
            "Year": data.get("Year"),
            "Genre": data.get("Genre"),
            "Director": data.get("Director"),
            "Actors": data.get("Actors"),
            "imdbRating": data.get("imdbRating"),
            "Plot": data.get("Plot")
        }
    else:
        return None

# 4️⃣ Consultar OMDb para todas las películas y guardar resultados
info_peliculas = []
for index, row in movies.iterrows():
    titulo = row["clean_title"]
    info = obtener_info_omdb(titulo)
    if info:
        info["movieId"] = row["movieId"]
        info_peliculas.append(info)
    time.sleep(0.2)  # para no saturar la API (limite de peticiones)

# 5️⃣ Crear DataFrame final
df_omdb = pd.DataFrame(info_peliculas)
##df_omdb.to_csv("movies_omdb.csv", index=False) Esto guarda un csv asi que hay que ver una ruta en caso de que vayamos a usarlo

df_omdb.head()

,Title,Year,Genre,Director,Actors,imdbRating,Plot,movieId
0,Toy Story,1995,"Animation, Adventure, Comedy",John Lasseter,"Tom Hanks, Tim Allen, Don Rickles",8.3,A cowboy doll is profoundly jealous when a new...,1
1,Jumanji,1995,"Adventure, Comedy, Family",Joe Johnston,"Robin Williams, Kirsten Dunst, Bonnie Hunt",7.1,A magic board game summons jungle perils and a...,2
2,Grumpier Old Men,1995,"Comedy, Romance",Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret",6.7,Just as John and Max resolve to save their bel...,3
3,Waiting to Exhale,1995,"Comedy, Drama, Romance",Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devine",6.0,"Based on Terry McMillan's novel, this film fol...",4
4,Father of the Bride Part II,1995,"Comedy, Family, Romance",Charles Shyer,"Steve Martin, Diane Keaton, Martin Short",6.1,George Banks must deal not only with his daugh...,5


## Dataset de Amazon de reseñas

In [14]:
import pandas as pd
import gzip

file_path = "../data/interim/movies.txt.gz"
data = []
current_review = {}
max_reviews = 10000  # cambiar según lo que quieras cargar

with gzip.open(file_path, "rt", encoding="latin-1") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith("product/productId:"):
            if current_review:
                data.append(current_review)
                if len(data) >= max_reviews:
                    break  # paramos al llegar al límite
                current_review = {}
            current_review["productId"] = line[len("product/productId: "):]
        elif line.startswith("review/userId:"):
            current_review["userId"] = line[len("review/userId: "):]
        elif line.startswith("review/profileName:"):
            current_review["profileName"] = line[len("review/profileName: "):]
        elif line.startswith("review/helpfulness:"):
            current_review["helpfulness"] = line[len("review/helpfulness: "):]
        elif line.startswith("review/score:"):
            try:
                current_review["score"] = float(line[len("review/score: "):])
            except:
                current_review["score"] = None
        elif line.startswith("review/time:"):
            try:
                current_review["time"] = int(line[len("review/time: "):])
            except:
                current_review["time"] = None
        elif line.startswith("review/summary:"):
            current_review["summary"] = line[len("review/summary: "):] if len(line) > len("review/summary: ") else ""
        elif line.startswith("review/text:"):
            current_review["text"] = line[len("review/text: "):] if len(line) > len("review/text: ") else ""

    if current_review and len(data) < max_reviews:
        data.append(current_review)

df = pd.DataFrame(data)
print(df.head())
print(f"Se cargaron {len(df)} reseñas")



    productId          userId                                 profileName  \
0  B003AI2VGA  A141HP4LYPWMSR            Brian E. Erland "Rainbow Sphinx"   
1  B003AI2VGA  A328S9RN3U5M68                                  Grady Harp   
2  B003AI2VGA  A1I7QGUDP043DG                   Chrissy K. McVay "Writer"   
3  B003AI2VGA  A1M5405JH9THP9                                golgotha.gov   
4  B003AI2VGA   ATXL536YX71TR  KerrLines "&#34;Movies,Music,Theatre&#34;"   

  helpfulness  score        time  \
0         7/7    3.0  1182729600   
1         4/4    3.0  1181952000   
2        8/10    5.0  1164844800   
3         1/1    3.0  1197158400   
4         1/1    3.0  1188345600   

                                             summary  \
0  "There Is So Much Darkness Now ~ Come For The ...   
1  Worthwhile and Important Story Hampered by Poo...   
2                      This movie needed to be made.   
3                  distantly based on a real tragedy   
4  "What's going on down in Juarez and s

In [15]:
df.head(10)

,productId,userId,profileName,helpfulness,score,time,summary,text
0,B003AI2VGA,A141HP4LYPWMSR,"Brian E. Erland ""Rainbow Sphinx""",7/7,3.0,1182729600,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic..."
1,B003AI2VGA,A328S9RN3U5M68,Grady Harp,4/4,3.0,1181952000,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...
2,B003AI2VGA,A1I7QGUDP043DG,"Chrissy K. McVay ""Writer""",8/10,5.0,1164844800,This movie needed to be made.,The scenes in this film can be very disquietin...
3,B003AI2VGA,A1M5405JH9THP9,golgotha.gov,1/1,3.0,1197158400,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...
4,B003AI2VGA,ATXL536YX71TR,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",1/1,3.0,1188345600,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess..."
5,B003AI2VGA,A3QYDL5CDNYN66,"abra ""a devoted reader""",0/0,2.0,1229040000,Pretty pointless fictionalization,The murders in Juarez are real. This movie is ...
6,B003AI2VGA,AQJVNDW6YZFQS,Charles R. Williams,3/11,1.0,1164153600,"This is junk, stay away","Mexican men are macho rapists, gangsters or in..."
7,B00006HAXW,AD4CDZK7D31XP,Anthony Accordino,64/65,5.0,1060473600,A Rock N Roll History Lesson,"Over the past few years, public television has..."
8,B00006HAXW,A3Q4S5DFVPB70D,Joseph P. Aiello,26/26,5.0,1041292800,A MUST-HAVE video if you grew up in the 50's...,I recvd this video (DVD version) as a Christma...
9,B00006HAXW,A2P7UB02HAVEPB,"""bruce_from_la""",24/24,5.0,1061164800,If You Like DooWop You Gotta Have This DVD,"Wow! When I saw this show on PBS--that was it,..."


## Datasets de Reseñas

tiene reseñas y se clasifican en sentimiento, pero no estan asociados a peliculas asi que las asignamos aleatoriamente

In [16]:
csv_path = '../data/interim/imdb-dataset/IMDB Dataset SPANISH.csv'

df_imdb = pd.read_csv(csv_path)
df_imdb.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo


In [17]:
import numpy as np

# Se le asigna un id del dataset de peliculas

# Número de reseñas
n_reviews = len(df_imdb)

# Lista de IDs de películas disponibles
movie_ids = movies_df['movieId'].tolist()

# Asignar aleatoriamente un movieId a cada reseña
df_imdb['movieId'] = np.random.choice(movie_ids, size=n_reviews)

# Revisar
df_imdb.head()

,Unnamed: 0,review_en,review_es,sentiment,sentimiento,movieId
0,0,One of the other reviewers has mentioned that ...,Uno de los otros críticos ha mencionado que de...,positive,positivo,121005
1,1,A wonderful little production. The filming tec...,Una pequeña pequeña producción.La técnica de f...,positive,positivo,147469
2,2,I thought this was a wonderful way to spend ti...,Pensé que esta era una manera maravillosa de p...,positive,positivo,195939
3,3,Basically there's a family where a little boy ...,"Básicamente, hay una familia donde un niño peq...",negative,negativo,252448
4,4,"Petter Mattei's ""Love in the Time of Money"" is...","El ""amor en el tiempo"" de Petter Mattei es una...",positive,positivo,132052
